In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [4]:
import pickle
with open('dataset_emotion_preprocessed.pickle', 'rb') as f:
   dataset_emotion = pickle.load(f)

In [5]:
tf_idf_vectorizer = TfidfVectorizer()
emotion_vectorized = tf_idf_vectorizer.fit_transform(dataset_emotion['processed'])

In [5]:

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(emotion_vectorized, dataset_emotion['emotion'])

KNeighborsClassifier(n_neighbors=3)

In [6]:

X_train, X_test, y_train, y_test = train_test_split(dataset_emotion['processed'], dataset_emotion['emotion'], test_size=0.20, random_state=42)


In [7]:
tf_idf_vectorizer = TfidfVectorizer()
emotion_vectorized = tf_idf_vectorizer.fit(dataset_emotion['processed'])

In [42]:
for i in range(1,11):    
    neigh = KNeighborsClassifier(n_neighbors=i)
    neigh.fit(tf_idf_vectorizer.transform(X_train), y_train)
    y_pred = neigh.predict(tf_idf_vectorizer.transform(X_test))
    print('k=',i,' ',accuracy_score(y_test,y_pred))

k= 1   0.2680274385143048
k= 2   0.23381294964028776
k= 3   0.2270369750711059
k= 4   0.23883219006190395
k= 5   0.22352350677597457
k= 6   0.21842061234733143
k= 7   0.2151581060732809
k= 8   0.20813116948301824
k= 9   0.23506775974569183
k= 10   0.27430148904132506
